#### 이미지 분류 예제(Gradio + gemma2 사용)

In [1]:
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image
import requests
from io import BytesIO
from langchain_community.chat_models import ChatOllama

c:\Python310\aiproject\llm2_Study\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# TensorFlow MobileNetV2 모델 로드
model = tf.keras.applications.MobileNetV2(weights="imagenet")  # 사전 훈련된 가중치를 사용

In [3]:
OLLAMA_SERVER = "http://localhost:11434"  # 로컬 서버 주소
MODEL_NAME = "gemma2"  # 사용하려는 Ollama 모델 이름

In [4]:
# Ollama를 사용해 음식 설명 생성
def get_food_description_with_langchain(food_name):
    """
    LangChain ChatOllama를 사용하여 음식 설명 생성
    """
    try:
        chat = ChatOllama(base_url=OLLAMA_SERVER, model=MODEL_NAME)
        prompt = f"{food_name}에 대해 특징, 효능, 요리 레시피 설명해줘.설명은 한국어로 해줘."
        response = chat.predict(prompt)
        return response
    except Exception as e:
        return f"Failed to retrieve description: {e}"

In [5]:
# 이미지 예측 함수
def predict_image_with_description(image_url):
    """
    이미지 URL을 받아 음식 예측과 Ollama 설명을 반환
    """
    try:
        # URL에서 이미지 가져오기
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content)).resize((224, 224))  # BytesIO 사용하여 이미지 열기

        # 이미지를 배열로 변환
        image_array = tf.keras.preprocessing.image.img_to_array(image)  # 이미지를 숫자 배열로 전환
        image_array = tf.expand_dims(image_array, axis=0)  # 모델이 한 번에 여러 이미지를 처리할 수 있게 "배치"라는 차원을 추가
        image_array = tf.keras.applications.mobilenet_v2.preprocess_input(image_array)  # 이미지 픽셀 값을 모델이 학습할 때 사용했던 범위로 조정 전처리

        # 예측 수행
        predictions = model.predict(image_array)
        decoded_predictions = tf.keras.applications.mobilenet_v2.decode_predictions(predictions, top=3)[0]  # 상위 3개 예측 결과 반환

        # 예측 결과 형식화
        result = {label: float(prob) for (_, label, prob) in decoded_predictions} # 예측 결과를 Gradio의 Label 컴포넌트가 요구하는 형식으로 변환

        # 가장 높은 확률의 예측값으로 Ollama 설명 생성
        top_food = decoded_predictions[0][1]  # 가장 확률이 높은 음식 이름
        description = get_food_description_with_langchain(top_food)

        return result, description  # 예측 결과와 Ollama 설명 반환

    except Exception as e:
        return {"error": 1.0}, f"Error: {e}"  # 에러 발생 시 기본값 반환

In [6]:
# Gradio 인터페이스 생성
iface = gr.Interface(
    fn=predict_image_with_description,
    inputs=gr.Textbox(label="이미지 URL 입력"),
    outputs=[
        gr.Label(num_top_classes=3, label="예측 결과"),  # 상위 3개 예측 결과
        gr.Textbox(label="음식 설명", interactive=False)  # Ollama로 생성한 설명 출력
    ],
    title="음식 이미지 분류 및 설명 생성기",
    description="이미지 URL을 입력하면 음식 분류 결과와 설명을 제공합니다."
)

In [7]:
# 인터페이스 실행
iface.launch(server_port=7861, server_name="0.0.0.0", debug=True)

* Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [8]:
iface.close()

Closing server running on port: 7861
